In [16]:
import pandas as pd

data_file = 'Goles & Mundiales.xlsx'
df = pd.read_excel(data_file, sheet_name='Datos_Original')

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2720 entries, 0 to 2719
Data columns (total 8 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   Fecha      2719 non-null   datetime64[ns]
 1   Local      2719 non-null   object        
 2   Visitante  2719 non-null   object        
 3   Anota      2719 non-null   object        
 4   Goleador   2720 non-null   object        
 5   Minutos    2719 non-null   float64       
 6   GC         2720 non-null   object        
 7   PEN        2720 non-null   object        
dtypes: datetime64[ns](1), float64(1), object(6)
memory usage: 170.1+ KB


#### Descripcion de los datos

+ Fecha: Indica la fecha del gol.

+ Local: Indica que el gol fue logrado por el equipo local.
+ Visitante: Indica que el gol fue logrado por el equipo visitante.
*Estas dos columnas fueron ignoradas en el trabajo*<sup><a href="#nota1">1</a></sup>.
+ Anota: Indica el equipo/pais que marco el gol.
+ Goleador: Indica el jugador que anota.
+ Minutos: Indica el minuto de tiempo reglamentario<sup><a href="#nota2">2</a></sup> en el que se anoto.
+ GC: Indica que el gol fue realizado en contra<sup><a href="#nota3">3</a></sup>.
+ PEN: Indica que el gol fue logrado de penal<sup><a href="#nota4">4</a></sup>.


<sup id="nota1">1</sup> En un Mundial de futbol, la condicion de local o visitante es arbitraria e irrelevante.  
<sup id="nota2">2</sup> Todos los goles son indicados dentro del tiempo reglamentario/suplementario. Por esto, los goles marcados en el añadido se informan como extension del reglamentario final. *Por ejemplo, si el gol se logra en el minuto 93 (90 + 3), se lo indica en el 90.*  
<sup id="nota3">3</sup> Es el unico caso donde el anotador es parte del equipo rival, pero suma para el que "Anota".  
<sup id="nota4">4</sup> Penal producto del juego, no de la serie para definir un partido. 

----
### Datos Estructurados

In [25]:
data_file = 'Goles & Mundiales.xlsx'
df = pd.read_excel(data_file, sheet_name='Datos_Estructurados')

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5438 entries, 0 to 5437
Data columns (total 7 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   Fecha      5438 non-null   datetime64[ns]
 1   Condicion  5438 non-null   object        
 2   Pais       5438 non-null   object        
 3   Goleador   5438 non-null   object        
 4   Minutos    5438 non-null   int64         
 5   Tiempo     5438 non-null   int64         
 6   Tipo       5438 non-null   int64         
dtypes: datetime64[ns](1), int64(3), object(3)
memory usage: 297.5+ KB


La transformacion de los datos fue lograda con Obtener & Transformar/PowerQuery de Excel. Se observan los siguientes cambios:

+ Condicion: Indica si el equipo posteriormente mencionado fue quien anoto o recibio el gol.
+ Tiempo: Ubica los minutos del gol dentro de un tiempo (1er o 2do Tiempo; 1er o 2do Suplementario)
+ Tipo: Indica el tipo de gol (Jugada, Penal o En Contra)

----
----
#### El Camino de los Datos

El proceso que sigue explica como llegue desde "Datos_Estructurados" hasta la base de datos que uso para crear mi presentacion en PowerBI.

El camino empezo con escasos conocimientos de Python, en donde no sabia de la existencia de las librerias Pandas ni NumPy. Asi que se puede observar ciertas dinamicas ineficientes quizas, pero que cumplen el proposito. Hoy podria hacerlas de forma diferente, pero eso espero se vera reflejado en futuros proyectos.  
En el caso de SQL (del que no sabia nada), agradezco a la formacion de CoderHouse, porque con lo visto pude hacer todo lo que quise para este proyecto. 

Si bien era posible importar los datos directamente desde el Excel mencionado, no habia desafio ni aprendizaje en eso. El Camino de los Datos es como sigue:

----
#### Parte 1: Desde Excel a SQL


El plan era cargar todo mi Excel en SQL mediante los queries adecuados. El problema estaba en la cantidad de datos: podia pasar años escribiendo cada uno. Lo resolvi automatizando la tarea con python.

In [30]:
import openpyxl as excel

file = "Goles & Mundiales.xlsx"
sql_file = "Camino de los Datos\Parte 1"

workbook = excel.load_workbook(file)
sheet = workbook["Datos_Estructurados"]

general = []
for index, row in enumerate(sheet.iter_rows()):
    
    general_temp = []
    if index > 0:
        row_data = [cell.value for cell in row]
        general_temp.append(str(row_data[0])[:10]) # Fecha

        # Aprovechando Python para la codificacion, se reemplazan las etiquetas por numeros:
        general_temp.append(row_data[1].replace('Anota','0').replace('Recibe','1')) # Condicion
        general_temp.append(row_data[2]) # Pais

        # Para evitar problemas con las '' en SQL, las transforme en * que luego seran reemplazados
        general_temp.append(row_data[3].replace("'","*")) # Goleador
        general_temp.append(row_data[4]) # Minutos
        general_temp.append(row_data[5]) # Tiempo
        general_temp.append(row_data[6]) # Tipo

        general.append(general_temp)

nombre_base = 'Goles'
nombre_tabla = 'General'
atributo_1 = 'Fecha'; atributo_2 = 'Condicion'; atributo_3 = 'FK_Pais'
atributo_4 = 'FK_Goleador'; atributo_5 = 'Minutos'; atributo_6 = 'FK_Tiempo'; atributo_7 = 'FK_Tipo'

printable_create = f'''
USE Goles

CREATE TABLE {nombre_tabla} (
    PK_{nombre_tabla} int PRIMARY KEY IDENTITY,
    {atributo_1} DATE, 
    {atributo_2} int,
    {atributo_3} VarChar(25),
    {atributo_4} VarChar(50),
    {atributo_5} int,
    {atributo_6} int,
    {atributo_7} int
    )
GO
'''

printable_insert = f'''
INSERT INTO {nombre_tabla}
    ({atributo_1}, {atributo_2}, {atributo_3}, {atributo_4}, {atributo_5}, {atributo_6}, {atributo_7})    
    VALUES
'''

printable_queries = ''
printable_queries += printable_create

for index, row in enumerate(general):

    if index == 0 or index % 1000 == 0:
        printable_queries += printable_insert

    printable_queries += (f"    ('{row[0]}', '{row[1]}', '{row[2]}', '{row[3]}', '{row[4]}', '{row[5]}', '{row[6]}')")

    if (index + 1) % 1000 != 0:
        printable_queries += ",\n"
    else:
        printable_queries += "\n"

printable_queries += f'\nGO\n\nSELECT * FROM {nombre_tabla}'

with open(f"{sql_file}.sql", 'w', encoding='utf-8') as file:
    file.write(printable_queries)

----
#### Parte 2: Paises & Continentes

##### Creacion de la Tabla "Paises"

+ Tome la columna "Pais" (Excel, Datos_Estructurados), la copie en una nueva hoja y elimine los duplicados. 
+ Agregue la columna "Codigo" y solicite a ChatGPT crear una lista con el continente de cada pais (suministre mi columna de paises como informacion).

Resultado:

In [32]:
data_file = 'Goles & Mundiales.xlsx'
df = pd.read_excel(data_file, sheet_name='Paises')

df

,Codigo,Paises,Continente
0,1,Algeria,África
1,2,Angola,África
2,3,Argentina,América del Sur
3,4,Australia,Oceanía
4,5,Austria,Europa
...,...,...,...
78,79,United Arab Emirates,Asia
79,80,United States,América del Norte
80,81,Uruguay,América del Sur
81,82,Wales,Europa


----
Creacion de los Queries para incorporar a la tabla a la base de datos SQL:

In [35]:
xlsx_file = "Goles & Mundiales.xlsx"
sql_file = "Camino de los Datos\Parte 2"

workbook = excel.load_workbook(xlsx_file)
sheet = workbook['Paises']

paises = []
for index, row in enumerate(sheet.iter_rows()):
    
    paises_temp = []
    if index > 0:
        row_data = [cell.value for cell in row]
        paises_temp.append(row_data[1])
        paises_temp.append(row_data[2])
    
        paises.append(paises_temp)

nombre_base = 'Goles'
nombre_tabla = 'Paises'
atributo_1 = 'Pais'; atributo_2 = 'FK_Continente'

printable_create = f'''
USE Goles

CREATE TABLE {nombre_tabla} (
    PK_Pais int PRIMARY KEY IDENTITY,
    {atributo_1} VarChar(50), 
    {atributo_2} VarChar(50),
    )
GO
'''

printable_insert = f'''
INSERT INTO {nombre_tabla}
    ({atributo_1}, {atributo_2})    
    VALUES
'''

printable_queries = ''
printable_queries += printable_create

for index, row in enumerate(paises):

    if index == 0 or index % 1000 == 0:
        printable_queries += printable_insert

    printable_queries += (f"    ('{row[0]}', '{row[1]}')")

    if (index + 1) % 1000 != 0:
        printable_queries += ",\n"
    else:
        printable_queries += "\n"

printable_queries += f'\nGO\n\nSELECT * FROM {nombre_tabla}'

with open(f"{sql_file}.sql", 'w', encoding='utf-8') as file:
    file.write(printable_queries)

----
Luego se procede con la codificacion:

In [36]:
xlsx_file = "Goles & Mundiales.xlsx"
sql_file = "Camino de los Datos\Parte 3"

workbook = excel.load_workbook(xlsx_file)
sheet = workbook['Paises']

table = 'General'
column = 'FK_Pais'

fechas= ''
for index, row in enumerate(sheet.iter_rows()):
    if index > 0:
        
        row_data = [cell.value for cell in row]
        fechas += f"UPDATE {table} SET {column} = {row_data[0]} WHERE {column} = '{row_data[1]}'\n"
        
with open(f"{sql_file}.sql", 'w', encoding='utf-8') as file:
    file.write(fechas)

----
Crear la tabla propia de continentes. El camino es similar:

1. Copiar la columna "Continente" de la tabla "Paises.
2. Eliminar duplicados
3. Agregar la columna de Indices.

In [34]:
data_file = 'Goles & Mundiales.xlsx'
df = pd.read_excel(data_file, sheet_name='Continentes')

df

,PK_Continente,Continente
0,1,África
1,2,América del Sur
2,3,Oceanía
3,4,Europa
4,5,América del Norte
5,6,Asia
6,7,América Central


Con dicha tabla, ya se pueden formar los queries de codificacion para la tabla "Paises"

In [37]:
xlsx_file = "Goles & Mundiales.xlsx"
sql_file = "Camino de los Datos\Parte 4"

workbook = excel.load_workbook(xlsx_file)
sheet = workbook['Continentes']

table = 'Paises'
column = 'FK_Continente'

fechas= ''
for index, row in enumerate(sheet.iter_rows()):
    if index > 0:
        
        row_data = [cell.value for cell in row]
        fechas += f"UPDATE {table} SET {column} = '{row_data[0]}' WHERE {column} = '{row_data[1]}'\n"
        
with open(f"{sql_file}.sql", 'w', encoding='utf-8') as file:
    file.write(fechas)

----
#### Parte 3: Goleadores

El camino es el mismo a las tablas anteriores: copiar la columna de interes, pegar en una nueva hoja y eliminar duplicados; crear columna de indices.  
Sin embargo, esta tabla tiene la particularidad de agregar a cada jugador el mundial donde mas goles anota "Mejor Mundial".

In [38]:
xlsx_file = "Goles & Mundiales.xlsx"
sql_file = "Camino de los Datos\Parte 5"

workbook = excel.load_workbook(xlsx_file)
sheet = workbook['Mejor Mundial_Data']

jugadores = []; lista_jugadores = {}

for index, row in enumerate(sheet.iter_rows()):
    if index > 0:
        row_data = [cell.value for cell in row]
        jugador = str(row_data[1]); mundial = str(row_data[0])[:4]

        if jugador not in lista_jugadores:
            lista_jugadores[jugador] = {}

        if mundial in lista_jugadores[jugador]:
            lista_jugadores[jugador][mundial] += 1
        else:
            lista_jugadores[jugador][mundial] = 1

mejores_mundiales = {}
for jugador, mundial in lista_jugadores.items():
    mundial_max = max(mundial, key=mundial.get)
    mejores_mundiales[jugador] = mundial_max

mejores_mundiales = dict(sorted(mejores_mundiales.items()))

printable = ''
instance = 1
for jugador, mejor_mundial in mejores_mundiales.items():
    jugador = jugador.replace("'","*")

    function_query = '''
INSERT INTO Goleadores
    (Goleador, Mundial)
    VALUES
'''

    create_query = '''USE Goles

CREATE TABLE Goleadores (
	PK_Goleadores int PRIMARY KEY IDENTITY,
	Goleador VarChar(50),
	Mundial int
	)

GO
'''
    if instance == 1:
        printable += create_query

    if instance % 1000 == 0 or instance == 1:
        printable += function_query
    printable += f"    ('{jugador}', {mejor_mundial})"
    if instance % 1000 != 0:
        printable += ',\n'

    instance += 1

with open(f"{sql_file}.sql", 'w', encoding='utf-8') as file:
    file.write(printable)

Codificacion de la informacion para adecuar la tabla General

In [40]:
xlsx_file = "Goles & Mundiales.xlsx"
sql_file = "Camino de los Datos\Parte 6"

workbook = excel.load_workbook(xlsx_file)
sheet = workbook['Jugadores']

table = 'General'
column = 'FK_Goleador'

fechas= ''
for index, row in enumerate(sheet.iter_rows()):
    if index > 0:
        
        row_data = [cell.value for cell in row]
        fechas += f"UPDATE {table} SET {column} = {row_data[0]} WHERE {column} = '{row_data[1]}'\n"
        
with open(f"{sql_file}.sql", 'w', encoding='utf-8') as file:
    file.write(fechas)

#### Parte 4: Otras consultas

Con lo visto hasta aqui, se explica en esencia como se formaron los queries para crear la base de datos en SQL.  
Sin embargo, fueron necesarias otras consultas que se pueden ya apreciar directamente en los ".sql" correspondientes. Asi:

+ "Goles & Mundiales.sql" describe con los comentarios correspondientes la creacion de la tabla "General" en PowerBI. Ademas, permite la creaciones de las tablas "Tipos", "Tiempo" y "Condicion".
+ "Goleadores.sql" la propia con dicha tabla en PowerBI.
+ "Paises.sql" la propia con dicha tabla en PowerBI, asi como la tabla "Continentes"
+ "Conexiones.sql" contiene los queries para establecer la relacion entra las tablas

Por lo tanto, para entender el proceso completo, se debera consultar dichos archivos, que contienen el proceso explicado mediante comentarios.